In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd

In [3]:
import altair as alt
import altair_stiles as altstiles

alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("stiles")

ThemeRegistry.enable('grid')

In [4]:
alt.renderers.set_embed_options(
    padding={"left": -10, "right": 0, "bottom": 0, "top": 0}
)

RendererRegistry.enable('default')

In [5]:
from oauth2client.service_account import ServiceAccountCredentials
import gspread

In [6]:
from df2gspread import df2gspread as d2g

In [7]:
from datawrapper import Datawrapper

dw = Datawrapper(
    access_token="FtIwtvFtoGLaRT9a3gjX69PLu4wSuRyKddoOz6SOPw3k9wWyNICMHTkcPhOGCR5Z"
)

In [8]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000

In [9]:
# pd.read_json("data/raw/layers/services5.arcgis.com/FDIcdEEqy1cBDX8N/Surf_Breaks_vs_Rising_Sea_Levels_CACACA_WFL1/Best Tide Conditions W/")

In [44]:
df = pd.read_json(
    "data/raw/layers/services5.arcgis.com/FDIcdEEqy1cBDX8N/Surf_Breaks_vs_Rising_Sea_Levels_CACACA_WFL1/Best Tide Conditions W/SLR_14ft_041m.geojson"
)

In [45]:
df.head()

,type,features
0,FeatureCollection,"{'type': 'Feature', 'id': 1, 'geometry': {'typ..."
1,FeatureCollection,"{'type': 'Feature', 'id': 2, 'geometry': {'typ..."
2,FeatureCollection,"{'type': 'Feature', 'id': 3, 'geometry': {'typ..."
3,FeatureCollection,"{'type': 'Feature', 'id': 4, 'geometry': {'typ..."
4,FeatureCollection,"{'type': 'Feature', 'id': 5, 'geometry': {'typ..."


In [46]:
missing_test = pd.json_normalize(df["features"])

In [47]:
missing_spots = missing_test[missing_test["properties.F0_41_value"].isnull()]

In [54]:
drowned_spots = missing_test[missing_test["properties.F0_41_text"] == "DROWNED"]

In [57]:
missing_spots

,type,id,geometry.type,geometry.coordinates,properties.OBJECTID,properties.No_,properties.County,properties.Break_,properties.Tide,properties.LatDec_Text,properties.FREQUENCY,properties.MEAN_BTideV,properties.COUNT_BTideV,properties.STD_BTideV,properties.Current_Best_Condition,properties.field_12,properties.F0_41_text,properties.F0_41_value,geometry
74,Feature,75,Point,"[-121.98166, 36.9575100000001]",75,47,Santa Cruz,26th Ave,Incoming to medium,36.95751,8,3.375000,8,0.517549,None,NaN,None,NaN,NaN
75,Feature,76,Point,"[-121.96848, 36.9578300000001]",76,49,Santa Cruz,38th Ave,Incoming to medium,36.95783,8,3.000000,8,0.755929,None,NaN,None,NaN,NaN
76,Feature,77,Point,"[-117.27364, 32.8133]",77,185,San Diego,Birdrock,varies medium-low to high,32.81330,3,3.666667,3,0.577350,None,NaN,None,NaN,NaN
77,Feature,78,Point,"[-121.93107, 36.6381000000001]",78,55,Monterey,Boneyard,Medium incoming,36.63810,1,3.000000,1,0.000000,None,NaN,None,NaN,NaN
78,Feature,79,Point,"[-119.29942, 34.27247]",79,98,Ventura,C Street (same as Ventura Point),Low to medium,34.27247,24,3.625000,24,0.824226,None,NaN,None,NaN,NaN
79,Feature,80,Point,"[-118.93265, 34.0441000000001]",80,102,Los Angeles,Leo Carrillo,Medium,34.04410,3,5.000000,3,0.000000,None,NaN,None,NaN,NaN
80,Feature,81,Point,"[-122.06113, 36.9490700000001]",81,41,Santa Cruz,Natural Bridges,Low,36.94907,2,3.500000,2,0.707107,None,NaN,None,NaN,NaN
81,Feature,82,Point,"[-117.93105, 33.6065100000001]",82,140,Orange,Newport Beach,x,33.60651,6,4.166667,6,0.752773,None,NaN,None,NaN,NaN
82,Feature,83,Point,"[-117.38665, 33.19392]",83,163,San Diego,Oceanside,Medium,33.19392,6,4.333333,6,1.032796,None,NaN,None,NaN,NaN
83,Feature,84,Point,"[-117.38544, 33.19129]",84,164,San Diego,Oceanside Pier,Medium,33.19129,5,4.600000,5,0.547723,None,NaN,None,NaN,NaN


In [37]:
counties = low_slr[["properties.OBJECTID", "properties.Break_", "properties.County"]]

In [46]:
surfspot_wcounties = surfspots.merge(counties, on="properties.OBJECTID")

In [75]:
county_surfspots = (
    surfspot_wcounties.groupby("properties.County")
    .size()
    .reset_index(name="surfspots_total")
    .rename(columns={"properties.County": "county"})
)

In [92]:
drowned_014m = (
    surfspot_wcounties.groupby(["properties.County"])["properties.F0_14_Text"]
    .apply(lambda x: x[x.str.contains("DROWNED")].count())
    .reset_index(name="drown_at_014m")
    .rename(columns={"properties.County": "county"})
)

In [93]:
surfspot_wcounties.head()

,type,id,geometry,properties.OBJECTID,properties.Surf_Break,properties.Current_Best_Condition,properties.field,properties.F0_14_Text,properties.F0_14_Value,properties.F0_41_text,properties.F0_41_value,properties.F0_69_text,properties.F0_69_value,properties.F0_96_text,properties.F0_96_Value,properties.F1_24_Text,properties.F1_24_Value,properties.F1_52_text,properties.F1_52_Value,properties.F1_65_Text,properties.F1_65_Value,properties.Break_,properties.County
0,Feature,1,None,1,Beacons,Medium Low,3.00,Low,2.50,Very Low,1.50,DROWNED,0.50,DROWNED,-0.50,DROWNED,-1.50,DROWNED,-2.50,DROWNED,-3.00,36th St Newport,Orange
1,Feature,2,None,2,Bird Rock South Bird,Medium,4.00,Medium Low,3.50,Low,2.50,Very Low,1.50,DROWNED,0.50,DROWNED,-0.50,DROWNED,-1.50,DROWNED,-2.00,54th/56th St Newport,Orange
2,Feature,3,None,3,Blacks,Medium Low,2.90,Low,2.40,Very Low,1.40,DROWNED,0.40,DROWNED,-0.60,DROWNED,-1.60,DROWNED,-2.60,DROWNED,-3.10,Asilomar,Monterey
3,Feature,4,None,4,Cardiff,Medium Low,3.08,Medium Low,2.58,Low,1.58,DROWNED,0.58,DROWNED,-0.42,DROWNED,-1.42,DROWNED,-2.42,DROWNED,-2.92,Avila Beach,San Luis Obispo
4,Feature,5,None,5,Coronado Beach,High,6.00,Medium high,5.50,Medium,4.50,Medium Low,3.50,Low,2.50,Very Low,1.50,DROWNED,0.50,DROWNED,0.00,Beacons,San Diego


In [96]:
drown_041m = (
    surfspot_wcounties.groupby(["properties.County"])["properties.F0_41_text"]
    .apply(lambda x: x[x.str.contains("DROWNED")].count())
    .reset_index(name="drown_at_041m")
    .rename(columns={"properties.County": "county"})
)

drown_069m = (
    surfspot_wcounties.groupby(["properties.County"])["properties.F0_69_text"]
    .apply(lambda x: x[x.str.contains("DROWNED")].count())
    .reset_index(name="drown_at_069m")
    .rename(columns={"properties.County": "county"})
)

drown_096m = (
    surfspot_wcounties.groupby(["properties.County"])["properties.F0_96_text"]
    .apply(lambda x: x[x.str.contains("DROWNED")].count())
    .reset_index(name="drown_at_096m")
    .rename(columns={"properties.County": "county"})
)

drown_124m = (
    surfspot_wcounties.groupby(["properties.County"])["properties.F1_24_Text"]
    .apply(lambda x: x[x.str.contains("DROWNED")].count())
    .reset_index(name="drown_at_124m")
    .rename(columns={"properties.County": "county"})
)

drown_152m = (
    surfspot_wcounties.groupby(["properties.County"])["properties.F1_52_text"]
    .apply(lambda x: x[x.str.contains("DROWNED")].count())
    .reset_index(name="drown_at_152m")
    .rename(columns={"properties.County": "county"})
)

drown_165m = (
    surfspot_wcounties.groupby(["properties.County"])["properties.F1_65_Text"]
    .apply(lambda x: x[x.str.contains("DROWNED")].count())
    .reset_index(name="drown_at_165m")
    .rename(columns={"properties.County": "county"})
)

### you don't have to have tool tips naming the surf spots, show them individually along the coast but don't let folks zoom in or see names/locations

In [98]:
drown_bycounty = county_surfspots.merge(drowned_014m, on="county")
drown_bycounty = drown_bycounty.merge(drown_041m, on="county")
drown_bycounty = drown_bycounty.merge(drown_069m, on="county")
drown_bycounty = drown_bycounty.merge(drown_096m, on="county")
drown_bycounty = drown_bycounty.merge(drown_124m, on="county")
drown_bycounty = drown_bycounty.merge(drown_152m, on="county")
drown_bycounty = drown_bycounty.merge(drown_165m, on="county")

In [100]:
drown_bycounty["pct_014m"] = 100 * (
    drown_bycounty["drown_at_014m"] / drown_bycounty["surfspots_total"]
)
drown_bycounty["pct_041m"] = 100 * (
    drown_bycounty["drown_at_041m"] / drown_bycounty["surfspots_total"]
)
drown_bycounty["pct_069m"] = 100 * (
    drown_bycounty["drown_at_069m"] / drown_bycounty["surfspots_total"]
)
drown_bycounty["pct_096m"] = 100 * (
    drown_bycounty["drown_at_096m"] / drown_bycounty["surfspots_total"]
)
drown_bycounty["pct_124m"] = 100 * (
    drown_bycounty["drown_at_124m"] / drown_bycounty["surfspots_total"]
)
drown_bycounty["pct_152m"] = 100 * (
    drown_bycounty["drown_at_152m"] / drown_bycounty["surfspots_total"]
)
drown_bycounty["pct_165m"] = 100 * (
    drown_bycounty["drown_at_165m"] / drown_bycounty["surfspots_total"]
)

In [102]:
drown_bycounty.to_csv("data/surfspots_bycounty.csv")

In [56]:
import geopandas as gpd

In [ ]:
gpd.read_file("data/raw/layers/")